In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ValidationZip.zip to ValidationZip.zip
Saving TestingZip.zip to TestingZip.zip
Saving TrainingZip.zip to TrainingZip.zip


Upload the zip files in the Hulth Data folder

In [ ]:
!unzip TrainingZip.zip

In [ ]:
!unzip TestingZip.zip

In [ ]:
!unzip ValidationZip.zip

In [ ]:
import os
PATH_TRAINING = "Training"
training_files = os.listdir(PATH_TRAINING)

PATH_VALIDATION = "Validation"
validation_files = os.listdir(PATH_VALIDATION)

PATH_TESTING = "Testing"
testing_files = os.listdir(PATH_TESTING)

In [ ]:
IDs = []
for i in training_files:
    ID = i.split(".")[0]
    ext = i.split(".")[1]
    if(ext == 'abstr'):
        IDs.append(int(ID))
IDs.sort()

In [ ]:
import re
fullText_TRAIN, keyWords_TRAIN  = [], []
for i in IDs:
    f1 = open(PATH_TRAINING+"/"+str(i)+".abstr",'r')
    f2 = open(PATH_TRAINING+"/"+str(i)+".uncontr",'r')
    fullText_TRAIN.append(f1.read().split())
    f2 = open(PATH_TRAINING+"/"+str(i)+".uncontr",'r')
    keyWords_TRAIN.append(f2.read().replace(";","").split())

In [ ]:
IDs = []
for i in validation_files:
    ID = i.split(".")[0]
    ext = i.split(".")[1]
    if(ext == 'abstr'):
        IDs.append(int(ID))
IDs.sort()

In [ ]:
import re
for i in IDs:
    f1 = open(PATH_VALIDATION+"/"+str(i)+".abstr",'r')
    f2 = open(PATH_VALIDATION+"/"+str(i)+".uncontr",'r')
    fullText_TRAIN.append(f1.read().split())
    f2 = open(PATH_VALIDATION+"/"+str(i)+".uncontr",'r')
    keyWords_TRAIN.append(f2.read().replace(";","").split())

In [ ]:
print(keyWords_TRAIN)

[['SBC', 'Communications', 'regulatory', 'compliance', 'telecom', 'carrier'], ['Sprint', 'telecom', 'service', 'providers', 'regulations', 'anti-spam', 'act', 'lawsuit'], ['Hutchison', 'Telecommunications', 'Singapore', 'Technologies', 'Global', 'Crossing', 'bankrupt'], ['competitive', 'capabilities', 'e-services', 'Internet', 'service', 'operations', 'strategy', 'business', 'performance', 'knowledge-based', 'economy', 'knowledge-based', 'competencies', 'human', 'capital', 'structural', 'capital', 'absorptive', 'capacity', 'operations-based', 'options', 'combinative', 'capabilities', 'service', 'quality', 'delivery', 'flexibility', 'cost', 'investment', 'intellectual', 'capital', 'bricks-and-mortar', 'clicks-and-mortar', 'dot-com', 'strategic', 'imperatives'], ['innovation', 'manufacturing', 'firms', 'business', 'network', 'assets', 'information', 'network', 'assets', 'research', 'network', 'assets', 'participation', 'assets', 'relational', 'assets', 'cognitive', 'social', 'capital', '

In [ ]:
IDs = []
for i in testing_files:
    ID = i.split(".")[0]
    ext = i.split(".")[1]
    if(ext == 'abstr'):
        IDs.append(int(ID))
IDs.sort()

In [ ]:
fullText_TEST, keyWords_TEST = [], []
for i in IDs:
    f1 = open(PATH_TESTING+"/"+str(i)+".abstr",'r')
    f2 = open(PATH_TESTING+"/"+str(i)+".uncontr",'r')
    fullText_TEST.append(f1.read().split())
    keyWords_TEST.append(f2.read().replace(";","").split())

In [ ]:
keyWordTags_TRAIN = []
for i in range(0,len(fullText_TRAIN)):
    tags = []
    for word in fullText_TRAIN[i]:
        if word in keyWords_TRAIN[i]:
            tags.append("TAG")
        else:
            tags.append("NO TAG")
    keyWordTags_TRAIN.append(tags)

In [ ]:
keyWordTags_TEST = []
for i in range(0,len(fullText_TEST)):
    tags = []
    for word in fullText_TEST[i]:
        if word in keyWords_TEST[i]:
            tags.append("TAG")
        else:
            tags.append("NO TAG")
    keyWordTags_TEST.append(tags)

In [ ]:
words = set([])
for i in range(0,len(fullText_TRAIN)):
    for word in fullText_TRAIN[i]: 
      try:
        words.add(word.lower())
      except:
        words.add("-OOV-")
      


In [ ]:
word2index = {w: i+2 for i,w in enumerate(list(words))}
word2index['-OOV-'] = 1
word2index['-PAD-'] = 0
tag2index = {'PAD':0,'TAG': 1, 'NO TAG':2}

In [ ]:
train_sentences_X, train_tags_y, test_sentences_X, test_tags_y = [], [], [], []
for s in fullText_TRAIN:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    train_sentences_X.append(s_int)

for s in keyWordTags_TRAIN:
    train_tags_y.append([tag2index[t] for t in s])

for s in fullText_TEST:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_sentences_X.append(s_int)

for s in keyWordTags_TEST:
    test_tags_y.append([tag2index[t] for t in s])

In [ ]:
MAX_LENGTH = len(max(train_sentences_X, key=len))

In [ ]:
from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'float32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'float32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

In [ ]:
from keras.preprocessing.sequence import pad_sequences
train_sentences_X = pad_sequences(train_sentences_X,maxlen=MAX_LENGTH,padding='post')
train_tags_y = pad_sequences(train_tags_y,maxlen=MAX_LENGTH,padding='post')

In [ ]:
test_sentences_X = pad_sequences(test_sentences_X,maxlen=MAX_LENGTH,padding='post')
test_tags_y = pad_sequences(test_tags_y,maxlen=MAX_LENGTH,padding='post')

In [ ]:
train_tags_y.shape

(1500, 502)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH,)))
model.add(Embedding(len(word2index),128,mask_zero=True))
model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(TimeDistributed(Dense(3)))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = Adam(0.001),
             metrics = ['accuracy',ignore_class_accuracy(0)])

In [ ]:
model.summary()

In [ ]:
import numpy as np
def to_categorical2(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [ ]:
model.fit(train_sentences_X,to_categorical2(train_tags_y,3),batch_size=128,epochs=50,validation_split=0.2)

In [ ]:
scores = model.evaluate(test_sentences_X, to_categorical2(test_tags_y, 3))

In [ ]:
print(scores)